## Pinterest Data Pipeline: Mounting S3 Bucket to Databricks

- Mount the desired S3 bucket to your Databricks account.
- Read in the data from the Delta table located at dbfs:/user/hive/warehouse/authentication_credentials.
- Ensure that when reading JSON files from S3, you include the complete path to the JSON objects, as observed in your S3 bucket (e.g., topics/<your_UserId>.pin/partition=0/).
- Create three different DataFrames:
  - df_pin for Pinterest post data
  - df_geo for geolocation data
  - df_user for user data.

In [ ]:
# pyspark functions
from pyspark.sql.functions import *
# URL processing
import urllib

In [ ]:
%run "/Users/<youremail>/read_access_keys"

In [ ]:
# # Define the path to the Delta table
# delta_table_path = "dbfs:/user/hive/warehouse/authentication_credentials"

# # Read the Delta table to a Spark DataFrame
# aws_keys_df = spark.read.format("delta").load(delta_table_path)

In [ ]:
# # Get the AWS access key and secret key from the spark dataframe
# ACCESS_KEY = aws_keys_df.select('Access key ID').collect()[0]['Access key ID']
# SECRET_KEY = aws_keys_df.select('Secret access key').collect()[0]['Secret access key']
# # Encode the secrete key
# ENCODED_SECRET_KEY = urllib.parse.quote(string=SECRET_KEY, safe="")

In [ ]:
# AWS S3 bucket name
AWS_S3_BUCKET = "user-0af0031518e7-bucket"
# Mount name for the bucket
MOUNT_NAME = "/mnt/mount_name"
# Source url
SOURCE_URL = "s3n://{0}:{1}@{2}".format(ACCESS_KEY, ENCODED_SECRET_KEY, AWS_S3_BUCKET)
# Mount the drive
dbutils.fs.mount(SOURCE_URL, MOUNT_NAME)

Out[102]: True

In [ ]:
display(dbutils.fs.ls("dbfs:/mnt/mount_name/topics/topics"))

path,name,size,modificationTime
dbfs:/mnt/mount_name/topics/topics/1209b9ad90a5.geo/,1209b9ad90a5.geo/,0,1708031778559
dbfs:/mnt/mount_name/topics/topics/1209b9ad90a5.pin/,1209b9ad90a5.pin/,0,1708031778559
dbfs:/mnt/mount_name/topics/topics/1209b9ad90a5.user/,1209b9ad90a5.user/,0,1708031778559


In [ ]:
%sql
-- Disable format checks during the reading of Delta tables
SET spark.databricks.delta.formatCheck.enabled=false

key,value
spark.databricks.delta.formatCheck.enabled,false


In [ ]:
# File location and type
file_type = "json"
# Ask Spark to infer the schema
infer_schema = "true"

# Define file paths for each dataframe
pin_file_location = "dbfs:/mnt/mount_name/topics/topics/1209b9ad90a5.pin/partition=0/*.json"
geo_file_location = "dbfs:/mnt/mount_name/topics/topics/1209b9ad90a5.geo/partition=0/*.json"
user_file_location = "dbfs:/mnt/mount_name/topics/topics/1209b9ad90a5.user/partition=0/*.json"

# Read in JSONs for each dataframe
df_pin = spark.read.format(file_type) \
    .option("inferSchema", infer_schema) \
    .load(pin_file_location)

df_geo = spark.read.format(file_type) \
    .option("inferSchema", infer_schema) \
    .load(geo_file_location)

df_user = spark.read.format(file_type) \
    .option("inferSchema", infer_schema) \
    .load(user_file_location)

# Display Spark dataframes to check their content
display(df_pin)
display(df_geo)
display(df_user)


category,description,downloaded,follower_count,image_src,index,is_image_or_video,poster_name,save_location,tag_list,title,unique_id
travel,"This Costa Rica itinerary is the ultimate guide to spending two weeks in Costa Rica. Find out about visiting La Fortuna, Arenal, Monteverde, Naranjo, Corcovado National Park, Or…",1,10k,https://i.pinimg.com/originals/30/93/cb/3093cb01d9de2d125fda8ba5e3e41946.jpg,10138,image,"Wanderlust Chloe ✈️ Travel guides, inspo and adventure travel ✈️",Local save in /data/travel,"Costa Rica Travel,Rio Celeste Costa Rica,Dream Vacations,Vacation Spots,Vacation Travel,Travel Pictures,Travel Photos,Fortuna Costa Rica,Costa Rica Pictures","14 Amazing Things To Do In Costa Rica | Volcanoes, Waterfalls, Wildlife And More",927c4658-cc3f-4b92-9b5c-70743d0c238d
diy-and-crafts,"This post may contain affiliate links, read our Disclosure Policy for more information. As an Amazon Associate I earn from qualifying purchases, thank you! Make some cute handpr…",1,892k,https://i.pinimg.com/originals/ff/fe/38/fffe384f3ec18a0d87cb2d80cc8c1499.jpg,3156,image,Michelle {CraftyMorning.com},Local save in /data/diy-and-crafts,"Christmas Gifts For Parents,Christmas Decorations For Kids,Christmas Crafts For Toddlers,Preschool Christmas,Christmas Crafts For Gifts,Christmas Activities,Toddler Crafts,Kids Christmas,Christmas Feeling",Handprint Reindeer Ornaments - Crafty Morning,fa6e31a4-18c2-4eca-a6d8-e903eee2c2a4
finance,"Der #Vermögensaufbau mit #Dividenden ist bereits mit 1.000 Euro möglich. Ja genau! Investieren ist nicht nur für Reiche etwas, sondern jeder kann mit bescheidenden Mitteln start…",1,3k,https://i.pinimg.com/originals/03/79/d1/0379d1df3ef5a5a492573c19205257db.jpg,5395,image,DIVDepot – Geld sparen & verdienen | Mindset | Dividend Investing,Local save in /data/finance,"Money Saving Challenge,Saving Money,Make Money Online,How To Make Money,Dividend Investing,Budget Planer,Discipline,Bulletins,Budgeting Finances",So startest du mit 1.000 Euro den Vermögensaufbau mit Dividenden - DIVDepot,71f72304-3708-45fb-8867-7c08995b10d2
finance,"If you love budgeting, make sure to give Dave Ramsey's 7 Baby Steps a try. Follow these steps to begin your debt snowball, build an emergency fund, invest and reach riches. I ca…",1,26k,https://i.pinimg.com/originals/1e/9d/90/1e9d906e4e150e3b95187f3b76ea7c71.png,5494,image,"Living Low Key | Save Money, Make Money, & Frugal Living",Local save in /data/finance,"Financial Peace,Financial Tips,Saving Money Quotes,Total Money Makeover,Budgeting Finances,Money Management,Wealth Management,Personal Finance,Making Ideas",Dave Ramsey's 7 Baby Steps: What Are They And Will They Work For You,8fb2af68-543b-4639-8119-de33d28706ed
christmas,Here are the best DIY Christmas Centerpieces ideas perfect for your Christmas & holiday season home decor. From Christmas Vignettes to Table Centerpieces.,1,500k,https://i.pinimg.com/originals/aa/6d/0f/aa6d0f44d7c1c96b998cb9aa6c4446b8.png,2418,image,HikenDip,Local save in /data/christmas,"Farmhouse Christmas Decor,Rustic Christmas,Christmas Time,Vintage Christmas,Xmas,Primitive Christmas Crafts,Christmas Vignette,Indoor Christmas Decorations,Diy Christmas Ornaments",100 DIY Christmas Centerpieces You'll Love To Decorate Your Home With For The Christmas Season - Hike n Dip,da8745a6-5160-46c4-877d-181d50a729fd
beauty,"[PaidLink] It's made with clarifying black clay and useless sea mud-plus natural almond and avocado oils to help moisturize and soothe your skin, while absorbing excess oils. An…",1,22,https://i.pinimg.com/originals/ff/09/71/ff097153a9d3fe3c9142c3cd7e9ed8d7.png,1611,image,Hair Accessories,Local save in /data/beauty,"Natural Beauty Tips,Health And Beauty Tips,Health Tips,Beauty Guide,Beauty Secrets,Natural Beauty Remedies,Skin Secrets,Natural Cures,Health Benefits","Over The Years, People Have Turn Into More Aware Of What's Inside Their Products And Have Begun T...",051e231c-7509-4379-9b9b-352883d63b8c
education,Tired of Google Classroom - need something a

country,ind,latitude,longitude,timestamp
Antarctica (the territory South of 60 deg S),2418,-88.4642,-171.061,2022-05-27 11:30:59
Bouvet Island (Bouvetoya),9270,-84.3984,-144.933,2022-05-23 03:24:24
Cocos (Keeling) Islands,10794,-89.5236,-154.567,2022-01-01 02:26:50
British Virgin Islands,5395,-6.93422,-24.9989,2020-08-15 19:30:19
Central African Republic,2074,-52.3213,-50.11,2019-11-03 05:41:59
Libyan Arab Jamahiriya,1170,56.1198,20.2963,2019-08-07 22:17:07
British Virgin Islands,9550,-84.918,7.23235,2022-03-23 00:25:45
Antigua and Barbuda,7922,-88.0974,-172.052,2021-01-27 09:14:19
Dominican Republic,9979,14.9967,-120.682,2018-07-18 19:01:46
Dominican Republic,6575,81.7192,61.5152,2020-12-29 22:59:37


age,date_joined,first_name,ind,last_name
20,2015-10-23 04:13:23,Alexandria,4249,Alvarado
21,2015-11-10 09:27:42,Andrea,8731,Alexander
21,2015-11-10 09:27:42,Andrea,8731,Alexander
21,2015-11-10 09:27:42,Andrea,8731,Alexander
34,2016-01-23 03:59:37,Daniel,10137,Matthews
21,2015-11-10 09:27:42,Andrea,8731,Alexander
24,2016-03-31 20:56:39,Austin,8887,Rodriguez
36,2015-12-20 16:38:13,Michelle,4315,Prince
32,2017-10-10 20:09:33,Christian,10625,Lang
36,2015-12-20 16:38:13,Michelle,4315,Prince


In [ ]:
## Step 5 (Optional): Unmount S3 bucket
If you want to unmount the S3 bucket, run the following code:

`dbutils.fs.unmount("/mnt/mount_name")`